<a href="https://colab.research.google.com/github/tomasborrella/TheValley/blob/main/notebooks/spark01/Ejercicio_resuelto_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio resuelto de PySpark

Notebook por [Tomás Borrella Martín](https://www.linkedin.com/in/tomasborrella/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/10HZGQnFNzRO63I9XRt-uQa6K9K2yAM71Wu-SYB0TL7c/edit?usp=sharing)

# Ejercicio propuesto

Crear un RDD con el log completo (no la versión simplificada) y contar las canciones usando los métodos "map" y "reduceByKey" del RDD

Pasos:
1. Descargar el archivo completo de la siguiente ruta:
https://raw.githubusercontent.com/tomasborrella/TheValley/main/data/spark01/complete_songs_log.txt
2. Crear un RDD de Spark con el contenido del archivo. (importante tener una sesión de Spark activa)
3. Usar los métodos del RDD para contar las reproducciones de las canciones.

Pista: Solo es necesario añadir un pequeño preprocesado en la fase de Map antes de devolver la tupla.

# 1. Datos
Descargamos un archivo que contiene una canción en cada fila (simplificado para el ejemplo).

NOTA: En un notebook, "!" ejecuta comandos del sistema desde dentro del notebook.

In [1]:
!wget -P /content/data 'https://raw.githubusercontent.com/tomasborrella/TheValley/main/data/spark01/complete_songs_log.txt' 

--2021-06-06 07:26:05--  https://raw.githubusercontent.com/tomasborrella/TheValley/main/data/spark01/complete_songs_log.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126125 (123K) [text/plain]
Saving to: ‘/content/data/complete_songs_log.txt’

complete_songs_log. 100%[===================>] 123.17K  --.-KB/s    in 0.02s   

2021-06-06 07:26:05 (5.55 MB/s) - ‘/content/data/complete_songs_log.txt’ saved [126125/126125]



# 2. Instalación Spark

In [2]:
# Install JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# Install Spark
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [4]:
# Install findspark
!pip install -q findspark

In [5]:
# Environment variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [6]:
# Find spark
import findspark
findspark.init()

In [7]:
# PySpark 
!pip install pyspark==3.1.1

     |████████████████████████████████| 212.3MB 60kB/s 
     |████████████████████████████████| 204kB 18.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=77f04ad559cace06feaef6dd06acb458d4b91452cc20669a0cd2411eafe9c369
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


# 3. Spark Session

In [8]:
# Imports
from pyspark.sql import SparkSession

In [9]:
# Create Spark Session
spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("Primer ejemplo con Spark")
         .getOrCreate()
)

In [10]:
# Show config
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1622964440267'),
 ('spark.driver.host', '7614e575bd0b'),
 ('spark.app.id', 'local-1622964441773'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'Primer ejemplo con Spark'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '36067'),
 ('spark.submit.deployMode', 'client'),
 ('spark.sql.warehouse.dir', 'file:/content/spark-warehouse'),
 ('spark.ui.showConsoleProgress', 'true')]

## RDD con los datos del log de canciones
(versión completa)

In [11]:
my_rdd = spark.sparkContext.textFile('/content/data/complete_songs_log.txt')

In [12]:
my_rdd.getNumPartitions()

2

In [13]:
type(my_rdd)

pyspark.rdd.RDD

In [14]:
pairs = my_rdd.map(lambda x: (x.split(",")[0], 1))

In [15]:
type(pairs)

pyspark.rdd.PipelinedRDD

In [16]:
pairs.collect()

[('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('The Box', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('The Box', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('The Box', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('The Box', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('The Box', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('The Box', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('The Box', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('Blinding Lights', 1),
 ('Dance Monkey', 1),
 ('Blinding Lights', 1),
 ('The Box', 1),
 ('Blinding 

In [17]:
result = pairs.reduceByKey(lambda x, y: x + y)
print(result.collect())

[('Blinding Lights', 1131), ('The Box', 510), ('Dance Monkey', 828)]


In [18]:
result.glom().collect()

[[('Blinding Lights', 1131), ('The Box', 510)], [('Dance Monkey', 828)]]

# Spark Stop

In [20]:
spark.stop()